<a href="https://colab.research.google.com/github/palanipsb/Chatbot/blob/master/TrainTicketBookingWithAdmin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
class TranDetails:
    trainData = {}

    def __init__(
        self,
        trainNo,
        trainName,
        seatCount1AC,
        baseFare1AC,
        seatCount2AC,
        baseFare2AC,
        seatCountSl,
        baseFareSl,
        seatCountG,
        baseFareG,
        stoppings,
        deptStat,
        arrivStat,
        pricePerKm,
    ):
        self.trainNo = trainNo
        self.trainName = trainName
        self.seatCount1AC = seatCount1AC
        self.baseFare1AC = baseFare1AC
        self.seatCount2AC = seatCount2AC
        self.baseFare2AC = baseFare2AC
        self.seatCountSl = seatCountSl
        self.baseFareSl = baseFareSl
        self.seatCountG = seatCountG
        self.baseFareG = baseFareG
        self.stoppings = stoppings
        self.deptStat = deptStat
        self.arrivStat = deptStat
        self.pricePerKm = pricePerKm

    def trainDataEntry(self):
        TranDetails.trainData.update(
            {
                self.trainNo: {
                    "name": self.trainName,
                    "availableCoaches": {
                        "1AC": {
                            "availableSeats": self.seatCount1AC,
                            "baseFare": self.baseFare1AC,
                        },
                        "2AC": {
                            "availableSeats": self.seatCount2AC,
                            "baseFare": self.baseFare2AC,
                        },
                        "SL": {
                            "availableSeats": self.seatCountSl,
                            "baseFare": self.baseFareSl,
                        },
                        "GN": {
                            "availableSeats": self.seatCountG,
                            "baseFare": self.baseFareG,
                        },
                    },
                    "stoppingSations": self.stoppings,
                    "departureStation": self.deptStat,
                    "arrivalStation": self.arrivStat,
                    "pricePerKm": self.pricePerKm,
                }
            }
        )


class StationDetails:
    stationData = {}
    stoppingTrains = {}

    def __init__(self, stationCode, stationName, distance, trainDetails):
        self.stationCode = stationCode
        self.stationName = stationName
        self.distance = distance
        self.trainDetails = trainDetails

    def stationDataEntry(self):
        stationData = {}
        for i, j in self.trainDetails.items():
            self.stoppingTrains.update(
                {
                    i: {
                        "arrivalTime": j["arrivalTime"],
                        "departureTime": j["departureTime"],
                    }
                }
            )
            StationDetails.stationData.update(
                {
                    self.stationCode: {
                        "name": self.stationName,
                        "distance": self.distance,
                        "stoppingTrainNo": self.stoppingTrains,
                    }
                }
            )


class FoodDetails:
    foodData = {}

    def __init__(self, trainNo, foodDetails):
        self.trainNo = trainNo
        self.foodDetails = foodDetails

    def foodDataEntry(self):
        self.cuisineList = {}
        for i, j in self.foodDetails.items():
            self.subFoodList = {}
            for k, l in j["mealOptions"].items():
                self.subFoodList.update(
                    {
                        k: {
                            "meal": l["meal"],
                            "availableQty": l["availableQty"],
                            "price": l["price"],
                        }
                    }
                )
            self.cuisineList.update(
                {i: {"cuisine": j["cuisine"], "mealOptions": self.subFoodList}}
            )
        self.foodData.update({self.trainNo: self.cuisineList})


In [89]:
t1 = TranDetails(100001,'Madurai Express',100,50.00,150,40.00,300,30.00,140,10.00,['VM','TPJ'],'MS','MDU',2.00)
t1.trainDataEntry()

In [ ]:
s1 = StationDetails('MS','Chennai Egmore',0,{100001:{'arrivalTime':'10.00 AM','departureTime':'10:10 AM'},100004:{'arrivalTime':'12:00 PM', 'departureTime':'12:10 PM'}})
s1.stationDataEntry()

In [130]:
f1 = FoodDetails(100001,{1:{'cuisine':'South Indian','mealOptions': {1:{'meal':'Break Fast - Idly', 'availableQty':50,'price':50},
                                                      2:{'meal':'Lunch - Biriyani', 'availableQty':50,'price':50}}},
                         2:{'cuisine':'North Indian','mealOptions': {1:{'meal':'Break Fast - Poori', 'availableQty':50,'price':50},
                                                      2:{'meal':'Lunch - Chapathi', 'availableQty':50,'price':50}}},})
f1.foodDataEntry()
FoodDetails.foodData

{100001: {1: {'cuisine': 'South Indian',
   'mealOptions': {1: {'meal': 'Break Fast - Idly',
     'availableQty': 50,
     'price': 50},
    2: {'meal': 'Lunch - Biriyani', 'availableQty': 50, 'price': 50}}},
  2: {'cuisine': 'North Indian',
   'mealOptions': {1: {'meal': 'Break Fast - Poori',
     'availableQty': 50,
     'price': 50},
    2: {'meal': 'Lunch - Chapathi', 'availableQty': 50, 'price': 50}}}}}

In [104]:
testData = {100001:{1:{'cuisine':'South Indian',1:{'meal':'Break Fast - Idly','availableQty':50,'price':50.00}}}}
testData

{100001: {1: {'cuisine': 'South Indian',
   1: {'meal': 'Break Fast - Idly', 'availableQty': 50, 'price': 50.0}}}}

In [92]:
TranDetails.trainData

{100001: {'name': 'Madurai Express',
  'availableCoaches': {'1AC': {'availableSeats': 100, 'baseFare': 50.0},
   '2AC': {'availableSeats': 150, 'baseFare': 40.0},
   'SL': {'availableSeats': 300, 'baseFare': 30.0},
   'GN': {'availableSeats': 140, 'baseFare': 10.0}},
  'stoppingSations': ['VM', 'TPJ'],
  'departureStation': 'MS',
  'arrivalStation': 'MS',
  'pricePerKm': 2.0}}

In [99]:
StationDetails.stationData

{'MS': {'name': 'Chennai Egmore',
  'distance': 0,
  'stoppingTrainNo': {100001: {'arrivalTime': '10.00 AM',
    'departureTime': '10:10 AM'},
   100004: {'arrivalTime': '12:00 PM', 'departureTime': '12:10 PM'}}}}